In [3]:
!pip install -q transformers sentence-transformers spacy protobuf==3.20.3
!pip install -q torch torchvision torchaudio
!pip install -q torch-scatter torch-geometric

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel

In [37]:
pairs_df = pd.read_csv("/kaggle/input/datasets/mahbubahabib/final-try/openbook_qa_dataset.csv")
conceptnet_df = pd.read_csv("/kaggle/input/datasets/mahbubahabib/trycandy/conceptnet.en (1).csv", sep="\t")
edge_embeddings = np.load("/kaggle/input/datasets/mahbubahabib/edge-embedding/conceptnet_edge_embeddings (1).npy")

In [38]:
pairs_df.head(5)

,qid,question,label,correct_vector,answer,concepts,numbers,units,relations,cn_result,graph_edges
0,0,the sun is responsible for,0,"[0, 0, 0, 1]",puppies learning new tricks,"['responsible', 'sun', 'learn', 'puppy', 'tric...",[],[],[],"{'display': [{'head': 'small_dog', 'relation':...","[(40513, -3.6227420637112284), (2419135, -5.19..."
1,0,the sun is responsible for,0,"[0, 0, 0, 1]",children growing up and getting old,"['responsible', 'get', 'child', 'old', 'grow',...",[],[],[],"{'display': [{'head': 'sun', 'relation': 'used...","[(2482157, -1.2459407787876446), (81806, -2.03..."
2,0,the sun is responsible for,0,"[0, 0, 0, 1]",flowers wilting in a vase,"['responsible', 'vase', 'flower', 'sun', 'wilt']",[],[],[],"{'display': [{'head': 'wiltja', 'relation': 'r...","[(2175314, -2.5953154761253283), (1159448, -2...."
3,0,the sun is responsible for,1,"[0, 0, 0, 1]","plants sprouting, blooming and wilting","['responsible', 'sprout', 'bloom', 'plant', 's...",[],['m'],[],"{'display': [{'head': 'sun', 'relation': 'rela...","[(1989208, 1.5373568558815518), (2482163, 1.40..."
4,1,when standing miles away from mount rushmore,0,"[0, 0, 0, 1]",the mountains seem very close,"['rushmore', 'stand', 'mountain', 'mount', 'mi...",[],['m'],[],"{'display': [{'head': 'mount_rushmore', 'relat...","[(1523766, -1.9470868349075319), (2360942, -3...."


In [9]:
pairs_df=pairs_df.head(20)

In [39]:
pairs_df

,qid,question,label,correct_vector,answer,concepts,numbers,units,relations,cn_result,graph_edges
0,0,the sun is responsible for,0,"[0, 0, 0, 1]",puppies learning new tricks,"['responsible', 'sun', 'learn', 'puppy', 'tric...",[],[],[],"{'display': [{'head': 'small_dog', 'relation':...","[(40513, -3.6227420637112284), (2419135, -5.19..."
1,0,the sun is responsible for,0,"[0, 0, 0, 1]",children growing up and getting old,"['responsible', 'get', 'child', 'old', 'grow',...",[],[],[],"{'display': [{'head': 'sun', 'relation': 'used...","[(2482157, -1.2459407787876446), (81806, -2.03..."
2,0,the sun is responsible for,0,"[0, 0, 0, 1]",flowers wilting in a vase,"['responsible', 'vase', 'flower', 'sun', 'wilt']",[],[],[],"{'display': [{'head': 'wiltja', 'relation': 'r...","[(2175314, -2.5953154761253283), (1159448, -2...."
3,0,the sun is responsible for,1,"[0, 0, 0, 1]","plants sprouting, blooming and wilting","['responsible', 'sprout', 'bloom', 'plant', 's...",[],['m'],[],"{'display': [{'head': 'sun', 'relation': 'rela...","[(1989208, 1.5373568558815518), (2482163, 1.40..."
4,1,when standing miles away from mount rushmore,0,"[0, 0, 0, 1]",the mountains seem very close,"['rushmore', 'stand', 'mountain', 'mount', 'mi...",[],['m'],[],"{'display': [{'head': 'mount_rushmore', 'relat...","[(1523766, -1.9470868349075319), (2360942, -3...."
...,...,...,...,...,...,...,...,...,...,...,...
1195,298,what is an example of reproduction occurring i...,1,"[0, 0, 0, 1]",cats are unable to produce offspring until the...,"['occur', 'reproduction', 'adulthood', 'unable...",[],['m'],[],"{'display': [{'head': 'cat', 'relation': 'rela...","[(862400, -2.39248350177916), (1200578, -2.941..."
1196,299,"because of human logging activities, some anim...",0,"[0, 0, 0, 1]",air,"['activity', 'conserve', 'air', 'logging', 'hu...",[],['m'],['because'],"{'display': [{'head': 'air', 'relation': 'rela...","[(667227, -4.269134738727455), (1454364, -4.40..."
1197,299,"because of human logging activities, some anim...",0,"[0, 0, 0, 1]",water,"['activity', 'conserve', 'logging', 'human', '...",[],['m'],['because'],"{'display': [{'head': 'animal', 'relation': 'r...","[(691877, -5.171836304133568), (100268, -5.664..."
1198,299,"because of human logging activities, some anim...",0,"[0, 0, 0, 1]",legs,"['activity', 'leg', 'conserve', 'logging', 'hu...",[],['m'],['because'],"{'display': [{'head': 'human', 'relation': 're...","[(1294990, -3.2374331925565047), (691798, -4.1..."


In [40]:
import numpy as np
import ast

def safe_parse_edges(x):
    # if string from CSV → convert to list
    if isinstance(x, str):
        x = ast.literal_eval(x)
    return x

def softmax_normalize_edges(edge_list):
    if not edge_list:
        return []

    cleaned = []

    for e in edge_list:
        # case: (id, weight)
        if isinstance(e, (list, tuple)) and len(e) >= 2:
            eid = int(e[0])
            w = float(e[1])
            cleaned.append((eid, w))

        # case: (id,) or just id
        elif isinstance(e, (list, tuple)) and len(e) == 1:
            cleaned.append((int(e[0]), 0.0))

        elif isinstance(e, (int, np.integer)):
            cleaned.append((int(e), 0.0))

        else:
            continue

    if not cleaned:
        return []

    scores = np.array([w for (_, w) in cleaned], dtype=float)

    exp_scores = np.exp(scores)
    weights = exp_scores / exp_scores.sum()

    return [(cleaned[i][0], float(weights[i])) for i in range(len(cleaned))]

In [41]:
pairs_df["graph_edges"] = pairs_df["graph_edges"].apply(safe_parse_edges)
pairs_df["graph_edges_norm"] = pairs_df["graph_edges"].apply(softmax_normalize_edges)

In [42]:
pairs_df.to_csv("pairs_df_with_normalized_edges.csv", index=False)

In [64]:
conceptnet_df = conceptnet_df.rename(columns={
    "antonym": "relation",
    "ab_extra": "head",
    "ab_intra": "tail",
    "1.0": "weight"
})
conceptnet_df

,relation,head,tail,weight
0,antonym,ab_intra,ab_extra,1.0
1,antonym,abactinal,actinal,1.0
2,antonym,abandon,acquire,1.0
3,antonym,abandon,arrogate,1.0
4,antonym,abandon,embrace,1.0
...,...,...,...,...
2487804,usedfor,zoom_lens,procure_better_shot,1.0
2487805,usedfor,zoom_lens,see_things_bigger,1.0
2487806,usedfor,zoom_lens,seeing_distant_object_more_closely,1.0
2487807,usedfor,zoom_lens,take_pictures,1.0


In [27]:
relations = conceptnet_df["relation"].unique()
relation2id = {r:i for i,r in enumerate(relations)}
num_relations = len(relations)
num_relations

17

In [16]:
print(conceptnet_df.columns)

Index(['relation', 'head', 'tail', 'weight'], dtype='object')


In [51]:
import ast

edge_list = ast.literal_eval(edge_list)

In [52]:
print(type(edge_list))
print(type(edge_list[0]))

<class 'list'>
<class 'tuple'>


In [53]:
unique_qids = pairs_df["qid"].unique()

train_qids, test_qids = train_test_split(
    unique_qids,
    test_size=0.2,
    random_state=42
)

train_df = pairs_df[pairs_df["qid"].isin(train_qids)].reset_index(drop=True)
test_df  = pairs_df[pairs_df["qid"].isin(test_qids)].reset_index(drop=True)

In [54]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
text_model = AutoModel.from_pretrained("distilbert-base-uncased")

def encode_qa(question, answer):
    text = question + " " + answer
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = text_model(**inputs)
    return outputs.last_hidden_state[:,0,:]   # CLS

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [65]:
def build_subgraph(edge_list):

    node2id = {}
    edge_index = []
    edge_vecs = []
    rel_ids = []
    scores = []

    for edge_id, score in edge_list:

        edge_id = int(edge_id)
        row = conceptnet_df.iloc[edge_id]


        h = row["head"]
        r = row["relation"]
    
        t = row["tail"]

        if h not in node2id:
            node2id[h] = len(node2id)
        if t not in node2id:
            node2id[t] = len(node2id)

        edge_index.append([node2id[h], node2id[t]])
        edge_vecs.append(edge_embeddings[edge_id])
        rel_ids.append(relation2id[r])
        scores.append(score)

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous().to(device)
    edge_vecs = torch.tensor(edge_vecs, dtype=torch.float).to(device)
    rel_ids = torch.tensor(rel_ids, dtype=torch.long).to(device)
    scores = torch.tensor(scores, dtype=torch.float).to(device)

    return node2id, edge_index, edge_vecs, rel_ids, scores

NameError: name 'edge_id' is not defined

In [66]:
class GraphEncoder(nn.Module):
    def __init__(self, edge_dim, rel_dim, hidden_dim):
        super().__init__()

        self.rel_emb = nn.Embedding(num_relations, rel_dim)
        self.edge_linear = nn.Linear(edge_dim + rel_dim, hidden_dim)
        self.attn_linear = nn.Linear(hidden_dim, 1)

    def forward(self, edge_vecs, rel_ids, scores):

        rel_emb = self.rel_emb(rel_ids)
        edge_input = torch.cat([edge_vecs, rel_emb], dim=1)

        h = torch.relu(self.edge_linear(edge_input))

        # retrieval score based attention
        attn = torch.softmax(scores, dim=0).unsqueeze(1)

        graph_embedding = torch.sum(attn * h, dim=0)

        return graph_embedding

In [67]:
class QAGraphModel(nn.Module):
    def __init__(self, text_dim=768, graph_hidden=256):
        super().__init__()

        self.graph_encoder = GraphEncoder(
            edge_dim=edge_embeddings.shape[1],
            rel_dim=64,
            hidden_dim=graph_hidden
        )

        self.text_classifier = nn.Linear(text_dim, 1)
        self.final_fc = nn.Linear(text_dim + graph_hidden, 1)

    def forward(self, qa_embed, edge_vecs, rel_ids, scores):

        graph_embed = self.graph_encoder(edge_vecs, rel_ids, scores)

        combined = torch.cat([qa_embed.squeeze(0), graph_embed], dim=0)

        graph_score = self.final_fc(combined)
        text_score = self.text_classifier(qa_embed.squeeze(0))

        final_score = graph_score + text_score

        return final_score

In [68]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [69]:
model = QAGraphModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(3):

    model.train()

    for qid in train_df["qid"].unique():

        group = train_df[train_df["qid"] == qid]

        scores = []
        labels = []

        for _, row in group.iterrows():

            qa_embed = encode_qa(row["question"], row["answer"])

            node2id, edge_index, edge_vecs, rel_ids, edge_scores = \
                build_subgraph(row["graph_edges"])

            score = model(qa_embed, edge_vecs, rel_ids, edge_scores)
            scores.append(score)

            labels.append(row["label"])

        scores = torch.stack(scores).squeeze()
        label_index = torch.tensor(labels).argmax().unsqueeze(0).to(device)

        loss = criterion(scores.unsqueeze(0), label_index)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch done")

/tmp/ipykernel_55/2900223665.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  edge_vecs = torch.tensor(edge_vecs, dtype=torch.float).to(device)


Epoch done
Epoch done
Epoch done


In [70]:
pairs_df["prediction_score"] = 0.0

model.eval()

with torch.no_grad():

    for qid in test_df["qid"].unique():

        group = test_df[test_df["qid"] == qid]

        for idx, row in group.iterrows():

            qa_embed = encode_qa(row["question"], row["answer"])

            node2id, edge_index, edge_vecs, rel_ids, edge_scores = \
                build_subgraph(row["graph_edges"])

            score = model(qa_embed, edge_vecs, rel_ids, edge_scores)

            pairs_df.loc[idx, "prediction_score"] = score.item()

In [71]:
pairs_df

,qid,question,label,correct_vector,answer,concepts,numbers,units,relations,cn_result,graph_edges,graph_edges_norm,prediction_score
0,0,the sun is responsible for,0,"[0, 0, 0, 1]",puppies learning new tricks,"['responsible', 'sun', 'learn', 'puppy', 'tric...",[],[],[],"{'display': [{'head': 'small_dog', 'relation':...","[(40513, -3.6227420637112284), (2419135, -5.19...","[(40513, 0.20830584166068852), (2419135, 0.043...",-0.267738
1,0,the sun is responsible for,0,"[0, 0, 0, 1]",children growing up and getting old,"['responsible', 'get', 'child', 'old', 'grow',...",[],[],[],"{'display': [{'head': 'sun', 'relation': 'used...","[(2482157, -1.2459407787876446), (81806, -2.03...","[(2482157, 0.231205178598347), (81806, 0.10473...",-0.303057
2,0,the sun is responsible for,0,"[0, 0, 0, 1]",flowers wilting in a vase,"['responsible', 'vase', 'flower', 'sun', 'wilt']",[],[],[],"{'display': [{'head': 'wiltja', 'relation': 'r...","[(2175314, -2.5953154761253283), (1159448, -2....","[(2175314, 0.23859608654542905), (1159448, 0.2...",-0.295022
3,0,the sun is responsible for,1,"[0, 0, 0, 1]","plants sprouting, blooming and wilting","['responsible', 'sprout', 'bloom', 'plant', 's...",[],['m'],[],"{'display': [{'head': 'sun', 'relation': 'rela...","[(1989208, 1.5373568558815518), (2482163, 1.40...","[(1989208, 0.29155432343143867), (2482163, 0.2...",-0.282600
4,1,when standing miles away from mount rushmore,0,"[0, 0, 0, 1]",the mountains seem very close,"['rushmore', 'stand', 'mountain', 'mount', 'mi...",[],['m'],[],"{'display': [{'head': 'mount_rushmore', 'relat...","[(1523766, -1.9470868349075319), (2360942, -3....","[(1523766, 0.7337700755555124), (2360942, 0.10...",-0.369140
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,298,what is an example of reproduction occurring i...,1,"[0, 0, 0, 1]",cats are unable to produce offspring until the...,"['occur', 'reproduction', 'adulthood', 'unable...",[],['m'],[],"{'display': [{'head': 'cat', 'relation': 'rela...","[(862400, -2.39248350177916), (1200578, -2.941...","[(862400, 0.12441632451421626), (1200578, 0.07...",0.000000
1196,299,"because of human logging activities, some anim...",0,"[0, 0, 0, 1]",air,"['activity', 'conserve', 'air', 'logging', 'hu...",[],['m'],['because'],"{'display': [{'head': 'air', 'relation': 'rela...","[(667227, -4.269134738727455), (1454364, -4.40...","[(667227, 0.23783555207389875), (1454364, 0.20...",0.000000
1197,299,"because of human logging activities, some anim...",0,"[0, 0, 0, 1]",water,"['activity', 'conserve', 'logging', 'human', '...",[],['m'],['because'],"{'display': [{'head': 'animal', 'relation': 'r...","[(691877, -5.171836304133568), (100268, -5.664...","[(691877, 0.10820282052919979), (100268, 0.066...",0.000000
1198,299,"because of human logging activities, some anim...",0,"[0, 0, 0, 1]",legs,"['activity', 'leg', 'conserve', 'logging', 'hu...",[],['m'],['because'],"{'display': [{'head': 'human', 'relation': 're...","[(1294990, -3.2374331925565047), (691798, -4.1...","[(1294990, 0.3285593585255163), (691798, 0.127...",0.000000


In [72]:
pairs_df["pred"] = 0


In [73]:
for qid, group in pairs_df.groupby("qid"):
    
    # find index of max score in this group
    best_idx = group["prediction_score"].idxmax()
    
    # set that row prediction = 1
    pairs_df.loc[best_idx, "pred"] = 1

In [75]:
correct = 0
total = 0

for qid, group in pairs_df.groupby("qid"):
    
    pred_idx = group["prediction_score"].idxmax()
    
    if pairs_df.loc[pred_idx, "label"] == 1:
        correct += 1
    
    total += 1

accuracy = correct / total
print("Question-level Accuracy:", accuracy)

Question-level Accuracy: 0.33


In [76]:
accuracy = (pairs_df["pred"] == pairs_df["label"]).mean()
print("Accuracy:", accuracy)

Accuracy: 0.665
